# Assistants API 

assistant에게 메세지를 보내면 그 메세지에 따른 출력을 찾아주도록 할 수도 있고 그 도구를 통해서 모은 결과를 답변해줄 수도 있다. 
따라서 우리가 전에 봤던 Agent의 컨셉과 약간 비슷하지만 langchain을 이용해 agent를 구현했습니다. 

이번에는 Langchain을 사용하지 않고 이미 설치되어있는 OpenAI Python 패키지만을 사용할 것입니다. 

Assistant가 가지고 있는 4가지 부분에 주안점을 두자. 


1. Assistant 

2. Thread 

3. Messages 

4. Run 

추가 비용은 발생하지 않음. 

하지만 하위 모델을 사용할 때 GPT3, GPT4, Turbo는 따로 결제해야 합니다. 

